![DLI Header](images/DLI_Header.png)

# 살펴보기

## 작업 설명

- 주어진 문장과 자연어 쿼리에 대한 답변을 생성하게 하고자 합니다.
- 답변이 어떻게 생성되는가에 따라, 수행할 작업은 두 가지 부류로 나뉘게 됩니다:
    1. <b>추출식 질문 답변</b>
    2. 생성형 질문 답변

### S2S와 GPT 계열 모델들을 사용한 생성형 질의 응답

자연어로 주어진 질문과 문장에 대해 질문에 대한 답변을 생성합니다. BERT 계열 모델들과 다르게, 답변이 문장의 길이에 구애받지 않습니다.

In [ ]:
BRANCH = 'main'

# 불러오기와 상수들

In [ ]:
import os
import wget
import gc

import pytorch_lightning as pl
from omegaconf import OmegaConf

from nemo.collections.nlp.models.question_answering.qa_gpt_model import GPTQAModel
from nemo.collections.nlp.models.question_answering.qa_s2s_model import S2SQAModel

gc.disable()

In [ ]:
# 아래의 경로들을 설정합니다.
DATA_DIR = "data" # 데이터셋 저장 경로
WORK_DIR = "work_dir" # 학습된 모델들, 로그들, 추가적으로 다운로드 된 스크립트들을 저장하는 경로

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(WORK_DIR, exist_ok=True)

# 설정

모델은 다음과 같이 주요 섹션들을 선언하는 설정 파일에 의해 정의됩니다:
- **model**: 모델과 관련된 모든 선언들 - 언어 모델, 길이 예측, 최적화와 스케줄러들, 데이터셋들과 기타 관련 정보들
- **trainer**: PyTorch Lightning으로 전달할 선언들
- **exp_manager**: 실험 관리자를 설정하기 위한 모든 선언들 - 타겟 경로, 이름, 로그 기록 정보

기본 설정 파일은 `NeMo/examples/nlp/question_answering/conf/qa_conf.yaml`로 저장되며, 다른 모델들을 학습하기 위해 필요한 값들을 편집할 것입니다.

In [ ]:
# 모델의 기본 설정 파일 다운로드
config_dir = WORK_DIR + '/conf/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + "qa_conf.yaml"):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/question_answering/conf/qa_conf.yaml', config_dir)
else:
    print ('config file already exists')

In [ ]:
# this will print the entire default config of the model
config_path = f'{WORK_DIR}/conf/qa_conf.yaml'
print(config_path)
config = OmegaConf.load(config_path)
print("Default Config - \n")
print(OmegaConf.to_yaml(config))

# SQuAD v2.0으로 모델들을 학습시키고 테스트 하기

## 데이터셋

이번 예제에서, 우리는 [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) 데이터셋을 다운로드하여 어떻게 학습하고 추론하게 하는지 볼 것입니다. SQuAD1.0과 SQuAD 2.0의 두 개의 데이터셋이 있습니다. SQuAD 데이터셋의 이전 버전인 SQuAD 1.1은 500개 이상의 기사들에 대한 100,000개 이상의 질문-답변 쌍을 가지고 있습니다. SQuAD2.0 데이터셋은 SQuAD1.1의 100,000개의 질문들에 더하여, 대중들이 적대적으로 작성하여 답변 가능해보이지만 답변할 수 없는 50,000개 이상의 질문들이 포함되어 있습니다.

"squad" 경로에 학습과 평가를 위한 아래의 데 개의 파일들을 준비하였습니다:   

```
squad  
│
└───v1.1
│   │ -  train-v1.1.json
│   │ -  dev-v1.1.json
│
└───v2.0
    │ -  train-v2.0.json
    │ -  dev-v2.0.json
```

In [ ]:
!ls -LR {DATA_DIR}/squad

## 데이터셋 설정

In [ ]:
# 만약 True로 설정하면, 모델은 캐시 파일이 존재하면 형상을 불러오고, 캐시 파일이 없으면 형상 파일을 만들고 캐시 파일에 덮어씁니다.
config.model.dataset.use_cache = False

# 데이터셋에 답변할 수 없는 질문이 있는지 여부를 나타냅니다.
config.model.dataset.version_2_with_negative = True

# 데이터셋이 추출적 성격을 가지고 있는지를 나타냅니다.
# 만약 True라면, 답변을 포함하지 않는 문장 스팬/청크는 답변할 수 없는 것으로 취급됩니다.
config.model.dataset.check_if_answer_in_context = True

# 학습, 검증, 그리고 테스트 데이터셋의 경로를 설정합니다.
config.model.train_ds.file = f"{DATA_DIR}/squad/v2.0/train-v2.0.json"
config.model.validation_ds.file = f"{DATA_DIR}/squad/v2.0/dev-v2.0.json"
config.model.test_ds.file = f"{DATA_DIR}/squad/v2.0/dev-v2.0.json"

# 학습, 검증, 그리고 테스트 데이터셋의 배치 크기를 설정합니다.
config.model.train_ds.batch_size = 8
config.model.validation_ds.batch_size = 8
config.model.test_ds.batch_size = 8

# 데이터셋에서 사용할 샘플의 수를 설정합니다. -1로 설정하면 전체 데이터셋을 사용합니다.
config.model.train_ds.num_samples = 5000
config.model.validation_ds.num_samples = 1000
config.model.test_ds.num_samples = 100

## 학습기 설정하기

In [ ]:
config.trainer.max_epochs = 1
config.trainer.max_steps = -1 # max_epochs보다 우선권을 가집니다.
config.trainer.precision = 16
config.trainer.devices = [0] # CPU를 사용하려면 0, GPU들을 사용하려면 리시트를 쓰지만, 이번 튜토리얼은 여러 GPU들을 사용하지 않기 때문에 [0]으로 설정합니다. 만약 필요하다면 NeMo/examples/nlp/question_answering/question_answering.py를 사용하세요.
config.trainer.accelerator = "gpu"
config.trainer.strategy="auto"

## 실험 관리자 설정

In [ ]:
# config.exp_manager.exp_dir = WORK_DIR
# config.exp_manager.name = "QA-SQuAD2"
# config.exp_manager.create_wandb_logger=False

## SQuAD v2.0을 사용한 S2S BART 모델 

### 모델 설정

In [ ]:
# 사용할 언어 모델과 토크나이저를 설정합니다.
# 만약 tokenizer 이름이 제공되지 않았다면 토크나이저는 모델에서 가져옵니다.
config.model.language_model.pretrained_model_name = "facebook/bart-base"
config.model.tokenizer.tokenizer_name = "facebook/bart-base"

# 모델을 저장할 경로를 설정합니다.
config.model.nemo_path = f"{WORK_DIR}/checkpoints/bart_squad_v2_0.nemo"

config.exp_manager.create_checkpoint_callback = True

config.model.optim.lr = 5e-5

# gpt 모델에서 vocab_file을 제거합니다.
config.model.tokenizer.vocab_file = None

### 학습기를 생성하고 모델 초기화하기

In [ ]:
#Colab에서 tokenizer를 초기화할 때 오류가 발생하면 아래 줄을 주석 처리하고 실행하세요 (참조: https://github.com/huggingface/transformers/issues/8690)
# !rm -r /root/.cache/huggingface/

trainer = pl.Trainer(**config.trainer)
model = S2SQAModel(config.model, trainer=trainer)

### 모델 학습, 테스트, 그리고 저장하기

In [ ]:
trainer.fit(model)
trainer.test(model)

model.save_to(config.model.nemo_path)

### 저장된 모델을 불러오고 추론 시키기

In [ ]:
model = S2SQAModel.restore_from(config.model.nemo_path)

eval_device = [config.trainer.devices[0]] if isinstance(config.trainer.devices, list) else 1
model.trainer = pl.Trainer(
    devices=eval_device,
    accelerator=config.trainer.accelerator,
    precision=16,
    logger=False,
)

all_preds, all_nbest = model.inference(
    config.model.test_ds.file,
#     output_prediction_file=output_prediction_file,
#     output_nbest_file=output_nbest_file,
    num_samples=10, # -1로 설정하면 추론에 모든 샘플을 사용합니다.
)

for question_id in all_preds:
    print(all_preds[question_id])

## SQuAD v2.0를 사용한 GPT2 모델

### 실습 # 1 - 모델 설정

* `gpt2` 사전 학습 모델과 토크나이저를 사용하기 위해 `<FIXME>`를 수정하세요.

In [ ]:
# 사용할 언어 모델과 토크나이저를 설정합니다.
# 만약 tokenizer 이름이 제공되지 않았다면 토크나이저는 모델에서 가져옵니다.
config.model.language_model.pretrained_model_name = <<<<FIXME>>>>
config.model.tokenizer.tokenizer_name = <<<<FIXME>>>>

# 모델을 저장할 경로를 설정합니다.
config.model.nemo_path = f"{WORK_DIR}/checkpoints/gpt2_squad_v2_0.nemo"

config.exp_manager.create_checkpoint_callback = True

config.model.optim.lr = 1e-4

click ... to show solution. 

### Create trainer and initialize model

In [ ]:
trainer = pl.Trainer(**config.trainer)
model = GPTQAModel(config.model, trainer=trainer)

### 실습 # 2 - 학습기를 생성하고 모델 초기화하기

* 모델을 학습, 테스트 그리고 저장하기 위해 `<FIXME>`를 수정하세요.

In [ ]:
<<<<FIXME>>>>.fit(<<<<FIXME>>>>)
<<<<FIXME>>>>.test(<<<<FIXME>>>>)

<<<<FIXME>>>>.save_to(config.model.nemo_path)

click ... to show solution. 

### 실습 # 3 - 저장된 모델을 불러오고 추론 시키기

* 저장된 모델로부터 추론을 수행하기 위해 `<FIXME>`를 수정하세요.

In [ ]:
model = GPTQAModel.restore_from(config.model.nemo_path)

eval_device = [config.trainer.devices[0]] if isinstance(config.trainer.devices, list) else 1
model.trainer = pl.Trainer(
    devices=eval_device,
    accelerator=config.trainer.accelerator,
    precision=16,
    logger=False,
)

all_preds, all_nbest = model.<<<<FIXME>>>>(
    config.model.test_ds.file,
    num_samples=10, # -1로 설정하면 추론에 모든 샘플을 사용합니다.
)

for question_id in all_preds:
    print(all_preds[question_id])

click ... to show solution. 

![DLI Header](images/DLI_Header.png)